# Zadania – 3_2 LangChain Tools

*Prepared: 2025-10-04 10:00*

Definiowanie własnych narzędzi (tools) i integracja z agentami.

> **Instrukcja:** Zadania są krótkie i polegają na uzupełnieniu brakujących fragmentów kodu (`____`). Niektóre komórki zawierają komentarze `# TODO` i `# HINT`. Uruchamiaj komórki po kolei.

## 📦 Wymagania
- `langchain_core`, `langchain_openai`, `langchain` ≥ 0.2
- Opcjonalnie: `faiss-cpu`, `tiktoken`, `python-dotenv`

Jeśli nie masz klucza API, możesz uruchamiać komórki „na sucho” – celem jest *uzupełnienie kodu*, a nie koniecznie jego wykonanie.

### Zadanie 1: Proste narzędzie – kalkulator
Zaimplementuj funkcję dodawania i zarejestruj ją jako narzędzie LangChain.

In [ ]:

# TODO: Utwórz tool `add_two_numbers(a: float, b: float) -> float`
from langchain_core.tools import tool

@tool
def add_two_numbers(a: float, b: float) -> float:
    """Zwraca sumę dwóch liczb."""
    # uzupełnij ciało
    return a + b

print(add_two_numbers.invoke({"a": 2, "b": 3}))


### Zadanie 2: Narzędzie „mini‑wiki” (lokalna baza wiedzy)
Zbuduj narzędzie, które zwraca krótki opis po kluczu ze słownika.

In [ ]:

# TODO: Stwórz słownik i tool `mini_wiki(query: str)` zwracający opis lub komunikat o braku hasła
from langchain_core.tools import tool

DB = {
    "langchain": "Framework do budowy aplikacji z LLM (łańcuchy, pamięć, narzędzia).",
    "react": "ReAct łączy reasoning i acting agentów z narzędziami w pętli.",
    "rag": "RAG łączy retriever (wyszukiwanie) z generowaniem odpowiedzi."
}

@tool
def mini_wiki(query: str) -> str:
    """Zwróć krótki opis po kluczu (np. 'rag', 'langchain')."""
    return DB.get(query.lower().strip(), "Brak hasła w mini‑wiki.")

print(mini_wiki.invoke({"query": "RAG"}))


### Zadanie 3: Agent z narzędziami
Utwórz prostego agenta, który może wywołać `add_two_numbers` i `mini_wiki` w zależności od pytania.

In [ ]:
# TODO: Zbuduj agenta z zestawem tools i przetestuj 2 zapytania
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.prompts import ChatPromptTemplate

tools = [add_two_numbers, mini_wiki]

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Prosty prompt ReAct
prompt = ChatPromptTemplate.from_messages([
    ("system", "Jesteś agentem. Możesz używać narzędzi: add_two_numbers, mini_wiki. Odpowiadaj po polsku."),
    ("user", "{input}")
])

agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)
executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

print(executor.invoke({"input": "Dodaj 12.5 i 7.5"}))
print(executor.invoke({"input": "Wyjaśnij w jednym zdaniu co to RAG"}))
